# Image Classification 

## Libraries

In [ ]:
import time
import datetime
import pickle
import numpy as np
from sklearn import svm, metricsfrom
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from pandas import DataFrame

## Loading Dataset from the pickle

In [ ]:
# Start point
start_time = time.time()
print("Start time: ", datetime.datetime.now())

In [ ]:
pickle_in = open("Signature_Dataset_HOG.pickle","rb")
image_dataset = pickle.load(pickle_in)

## Split data in Training Set and Testing Set

In [ ]:
X_train = image_dataset.X_train
X_test = image_dataset.X_test
y_train = image_dataset.y_train
y_test = image_dataset.y_test

print("----Dataset Splitted----")

## Train data with parameter optimization

In [ ]:
#Dictionary of models and hyperparameters

models_and_hyperparameters={'LR':(LogisticRegression(),{'C': [0.01, 0.05, 0.1, 0.5, 1, 2],
                                                        'penalty': ['l1', 'l2'], 
                                                        'class_weight': ['balanced']}),
                           'SVM':(svm.SVC(),{'C': [1, 10, 100, 1000],
                                             'gamma': [0.001, 0.0001],
                                             'kernel': ['rbf'],
                                             'class_weight': ['balanced']}),
                           'RFC':(RandomForestClassifier(), {'n_estimators': [50,100,200],
                                                             # 'max_features': ['auto', 'sqrt'],
                                                             'max_depth': [50,60,70],
                                                             'min_samples_split': [2,5,10],
                                                             #'min_samples_leaf': [1,2],
                                                             'bootstrap': [True, False],
                                                             'class_weight': ['balanced']})
                           }

In [ ]:
#LOGISTIC REGRESSION

print("----Training----")

k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=31)

model = models_and_hyperparameters['LR'][0]
hyperparameters = models_and_hyperparameters['LR'][1]

gs = GridSearchCV(model, 
                   param_grid = hyperparameters,
                   cv=k_fold, 
                   scoring='accuracy',
                   verbose=3,
                   return_train_score=True)

gs.fit(X_train, y_train)

print("----Training Ended----")


In [ ]:
# SVM

print("----Training----")

k_fold = StratifiedKFold(n_splits=4, shuffle=True, random_state=31)

model = models_and_hyperparameters['SVM'][0]
hyperparameters = models_and_hyperparameters['SVM'][1]

gs = GridSearchCV(model, 
                   param_grid = hyperparameters,
                   cv=k_fold, 
                   scoring='accuracy',
                   verbose=3,
                   return_train_score=True)

gs.fit(X_train, y_train)

print("----Training Ended----")


In [ ]:
# RANDOM FOREST

print("----Training----")

k_fold = StratifiedKFold(n_splits=4, shuffle=True, random_state=31)

model = models_and_hyperparameters['RFC'][0]
hyperparameters = models_and_hyperparameters['RFC'][1]

gs = GridSearchCV(model, 
                   param_grid = hyperparameters,
                   cv=k_fold, 
                   scoring='accuracy',
                   verbose=3,
                   return_train_score=True)

gs.fit(X_train, y_train)

print("----Training Ended----")


In [ ]:
DataFrame(gs.cv_results_)

## Predict

In [ ]:
y_pred = gs.predict(X_test)

In [ ]:
print("Best hyperparameters: {}".format(gs.best_params_))
print("Best accuracy score: {:3f}".format(gs.best_score_))

## Report

In [ ]:
print("Classification report for - \n{}:\n{}\n".format(gs, metrics.classification_report(y_test, y_pred)))

In [ ]:
# End point
end_time = time.time()
uptime = end_time - start_time
human_uptime = datetime.timedelta(seconds=uptime)

print("End time: ", datetime.datetime.now())
print("Uptime :" ,human_uptime)